###### References: 
- Fluent Python, 2nd Edition, by Luciano Ramalho.

# Chapter 13: Interfaces, Protocols, and ABCs

<img src="Typing.png" width="75%">

## Two Kinds of Protocols

Compares the two forms of structural typing with protocols.

In [1]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

In [2]:
v = Vowels()
v[0]

'A'

In [3]:
v[-1]

'U'

In [4]:
for c in v: print(c)

A
E
I
O
U


In [5]:
'E' in v

True

In [6]:
'Z' in v

False

### Dynamic protocol
Informal protocols in Python.
### Static protocol
PEP 544 - Protocols: Structural subtyping

An object may implement only part of a dynamic protocol; but to fufill a static protocol, the object must provide every method declared in the protocol class.

Static protocols can be verified by static type checkers, but dynamic protocols can't.

## Programming Ducks
### Python Digs Sequences
<img src="Sequence.png" width="75%">


In [7]:
import collections

In [8]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [9]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

### Monkey Patching: Implementing a Protocol at Runtime

In [10]:
from random import shuffle

In [11]:
l = list(range(10))
shuffle(l)
l

[9, 2, 0, 3, 6, 1, 5, 8, 4, 7]

In [12]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [13]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [14]:
FrenchDeck.__setitem__ = set_card

shuffle(deck)
deck[:5]

[Card(rank='6', suit='clubs'),
 Card(rank='3', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='9', suit='clubs'),
 Card(rank='5', suit='clubs')]

### Defensive Programming and "Fail Fast"
    def __init__(self, iterable):
        self._balls = list(iterable)
        
`stflib/3/collections/__init__.pyi` :

    def namedtuple(
        typename: str,
        field_names: Union[str, Iterable[str]],
        *,
        # rest ommited
    
Duck typing to handle a string or an iterable of strings

    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass
    field_names = tuple(field_names)
    if not all(s.isidentifier() for s in field_names):
        raise ValueError('field_names must all be valid identifiers')


## Goose Typing
Subclassing from ABC's to make it explicit that you are implementing a previously defined interface.

Runnign type checking using ABC's instead of concrete classes as the second argument for `isinstance` and `issubclass`.

### Subclassing an ABC


In [15]:
from collections import namedtuple, abc

In [16]:
Card = namedtuple('Card', ['rank', 'suit'])

In [17]:
class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)

<img src='MutableSequence.png' width="75%">

### ABC in the Standard Library
<img src='ABC.png' width="75%">

### Defining and Using the ABC
<img src='Tombola.png' width="75%">

In [18]:
import abc

In [19]:
class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(items)

In [20]:
class Fake(Tombola):
    def pick(self):
        return 13

In [21]:
Fake

__main__.Fake

In [22]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

### ABC Syntax Details
    class MyABC(abc.ABC):
        @classmethod
        @abc.abstractmethod
        def an_abstract_classmethod(cls, ...):
            pass

### Subclassing an ABC

In [23]:
class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <6>
        self.pick()

In [24]:
class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # <1>

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  # <3>

    def loaded(self):  # <4>
        return bool(self._balls)

    def inspect(self):  # <5>
        return tuple(self._balls)

### A Virtual Sublass of an ABC
<img src="VirtualABC.png" width="50%">

In [25]:
from random import randrange

In [26]:
@Tombola.register  # <1>
class TomboList(list):  # <2>

    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5>

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(self)


In [27]:
issubclass(TomboList, Tombola)

True

In [28]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [29]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [30]:
BingoCage.__mro__

(__main__.BingoCage, __main__.Tombola, abc.ABC, object)

### Usage of register in Practice

    Sequence.register(tuple)
    Sequence.register(str)
    Sequence.register(range)
    Sequence.register(memoryview)

### Structural Typing with ABCs

In [31]:
class Struggle:
    def __len__(self): return 23

In [32]:
from collections import abc

In [33]:
isinstance(Struggle(), abc.Sized)

True

In [34]:
issubclass(Struggle, abc.Sized)

True

from `Lib/_collections_abc.py`

    class Sized(metaclass=ABCMeta):
        __slots__=()

        @abstractmethod
        def __len__(self):
            return 0

        @classmethod
        def __subclasshook__(cls, C):
            if cls is Sized:
                if any("__len__" in B.__dict__ for B in C.__mro__):
                    return True
            return NotImplemented

## Static Protocols
### The Typed double Function

In [35]:
def double(x):
    return x * 2

In [36]:
double(1.5)

3.0

In [37]:
double('A')

'AA'

In [38]:
double([10, 20, 30])

[10, 20, 30, 10, 20, 30]

In [39]:
from fractions import Fraction

In [40]:
double(Fraction(2,5))

Fraction(4, 5)

In [41]:
from typing import TypeVar, Protocol

In [42]:
T = TypeVar('T')

In [43]:
class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ... 

In [44]:
RT = TypeVar('RT', bound=Repeatable)

In [45]:
def double(x: RT) -> RT:  # <4>
    return x * 2

### Runtime Checkable Static Protocols

    class typing.SupportsComplex

An ABC with one abstract method, `__complex__`.

    class typing.SupportsFloat
    
An ABC with one abstract method, `__float__`.


    @runtime_checkable
    class SupportsComplex(Protocol):
        """An ABC with one abstract method __complex__."""
        __slots__ = ()
        
        @abstractmethod
        def __complex__(self) -> complex:
            pass

In [46]:
from typing import SupportsComplex
import numpy as np

In [47]:
c64 = np.complex64(3+4j)
isinstance(c64, complex)

False

In [48]:
isinstance(c64, SupportsComplex)

True

In [49]:
c = complex(c64)
c

(3+4j)

In [50]:
isinstance(c, SupportsComplex)

False

In [51]:
complex(c)

(3+4j)

In [52]:
isinstance(c, (complex, SupportsComplex))

True

In [53]:
import numbers

In [54]:
isinstance(c, numbers.Complex)

True

In [55]:
isinstance(c64, numbers.Complex)

True

### Limitations of Runtime Protocol Checks

In [56]:
import sys

In [57]:
sys.version

'3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]'

In [58]:
c = 3+4j

In [59]:
c.__float__()

AttributeError: 'complex' object has no attribute '__float__'

In [60]:
from typing import SupportsFloat

In [61]:
isinstance(c, SupportsFloat)

False

In [62]:
issubclass(complex, SupportsFloat)

False

### Supporting a Static Protocol

In [63]:
from array import array
import math

In [64]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V4_COMPLEX[]
    def __complex__(self):
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum):
        return cls(datum.real, datum.imag)

In [65]:
from typing import SupportsComplex, SupportsAbs

In [66]:
v = Vector2d(3, 4)
isinstance(v, SupportsComplex)

True

In [67]:
isinstance(v, SupportsAbs)

True

In [68]:
complex(v)

(3+4j)

In [69]:
abs(v)

5.0

In [70]:
Vector2d.fromcomplex(3+4j)

Vector2d(3.0, 4.0)

In [71]:
from typing import SupportsComplex, Iterator

In [72]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self) -> float:
        return self.__x

    @property
    def y(self) -> float:
        return self.__y

    def __iter__(self) -> Iterator[float]:
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self) -> str:
        return str(tuple(self))

    def __bytes__(self) -> bytes:
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other) -> bool:
        return tuple(self) == tuple(other)

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)

    def __bool__(self) -> bool:
        return bool(abs(self))

    def angle(self) -> float:
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec='') -> str:
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets) -> Vector2d:
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# tag::VECTOR2D_V5_COMPLEX[]
    def __abs__(self) -> float:  # <1>
        return math.hypot(self.x, self.y)

    def __complex__(self) -> complex:  # <2>
        return complex(self.x, self.y)

    @classmethod
    def fromcomplex(cls, datum: SupportsComplex) -> Vector2d:  # <3>
        c = complex(datum)  # <4>
        return cls(c.real, c.imag)

### Designing a Static Protocol

In [73]:
from typing import Protocol, runtime_checkable, Any

In [74]:
@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

In [75]:
from typing import Any, Iterable, TYPE_CHECKING

In [76]:
class SimplePicker:  # <2>
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:  # <3>
        return self._items.pop()

def test_isinstance() -> None:  # <4>
    popper: RandomPicker = SimplePicker([1])  # <5>
    assert isinstance(popper, RandomPicker)  # <6>

def test_item_type() -> None:  # <7>
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)  # <8>
    assert isinstance(item, int)

In [77]:
!mypy randompick_test.py

randompick_test.py:24: note: Revealed type is "Any"
Success: no issues found in 1 source file


### Extending a Protocol

In [78]:
from typing import Protocol, runtime_checkable

In [79]:
@runtime_checkable  # <1>
class LoadableRandomPicker(RandomPicker, Protocol):  # <2>
    def load(self, Iterable) -> None: ... 

### The numbers ABCs and Numeric Protocols
    _Number = Union[float, Decimal, Fraction]
    _NumberT = TypeVar('_NumberT', float, Decimal, Fraction)

In [80]:
cd = np.cdouble(3+4j)
cd

(3+4j)

In [81]:
float(cd)

/var/folders/9k/760_q9mj44x7wl8kzf8kxf3m0000gn/T/ipykernel_9458/3810435258.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  float(cd)


3.0

In [82]:
sample = [1+0j, np.complex64(1+0j), 1.0, np.float16(1.0), 1, np.uint8(1)]
[isinstance(x, SupportsComplex) for x in sample]

[False, True, False, False, False, False]

In [83]:
[complex(x) for x in sample]

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]